In [2]:
import numpy as np
from matplotlib import pyplot as plt
from functools import partial

def util_func_template(h, limit, slope):
    """
    limit is the maximum value of the function (as h approaches infinity)
    slope is the derivative at 0
        it loosely corresponds to the inverse of desired leverage
    """
    scaler = limit / (np.pi / 2)
    return scaler * np.arctan(h * slope / scaler)

In [18]:
def find_stable_payments(funcs, exponent=0.5):
    """
    receives a list of functions, where each one tells for a given person
    how much will they pay, given what all the rest is paying
    
    then, it finds a stable amount of payments for each person
    and returns it as a list
    
    default exponent is 0.5, which corresponds to quadratic funding
        - each payout is square rooted, and then all are summed up
        for exponent=1, we just sum up all the payouts,
        so then, we don't have to worry about sybil attacks
    """
    payouts = np.ones(len(funcs))

    change_size = float("inf")
    while change_size > 0.000001:
        h = sum(payouts ** exponent)
        new_payouts = np.array([func(h) for func in funcs])
        change_size = sum(np.abs(new_payouts - payouts))
        payouts = new_payouts

        # print(h)
        # print(payouts)
        # print()
    return payouts

In [19]:
def find_instant_leverages(funcs):
    epsilon = 0.001
    payouts = find_stable_payments(funcs)

    # find leverages for each person
    leverages = []
    for index in range(len(funcs)):
        new_funcs = funcs.copy()
        # modify the fixed value and see how it affects the pot
        new_funcs[index] = lambda h: payouts[index] + epsilon
        new_payouts = find_stable_payments(new_funcs)
        
        pot_difference = (sum(new_payouts) - sum(payouts))
        leverage = pot_difference / epsilon
        leverages.append(leverage)
    return np.array(leverages)

In [21]:
common_slope = 0.4
funcs = [
    partial(util_func_template, limit=10, slope=common_slope), 
    partial(util_func_template, limit=10, slope=common_slope), 
    partial(util_func_template, limit=10, slope=common_slope), 
    partial(util_func_template, limit=10, slope=common_slope), 
    partial(util_func_template, limit=1, slope=common_slope), 
]

payouts = find_stable_payments(funcs)
leverages = find_instant_leverages(funcs)
# print info
np.set_printoptions(formatter=dict(float=lambda num: f"{num:5.2f}"))
leverages = np.array(leverages)
print("payouts:   ", payouts)
print("leverages: ", leverages)
print("total pot: ", sum(payouts))

payouts:    [ 2.87  2.87  2.87  2.87  0.87]
leverages:  [ 1.41  1.41  1.41  1.41  2.12]
total pot:  12.361515820310577


# This section is work in progress

In [23]:
# try to find the slopes for which we attain the desired leverages
# note: these leverages aren't real now!

In [24]:
target_leverages = [1.5, 1.5, 1.5, 1.5, 1.5, 1.5]
limits = [10, 10, 10, 10, 10, 10]

slopes = np.ones(len(limits))
# slopes = [1,1,1,1]

In [25]:
for _ in range(100):
    # construct functions given limits and slopes
    funcs = [
        partial(util_func_template, limit=limit, slope=slope) 
        for limit, slope in zip(limits, slopes)
    ]

    leverages = find_instant_leverages(funcs)
    payouts = find_stable_payments(funcs)

    # update slopes to be closer to the target leverages
    alpha = 0.3
    gradients = - (target_leverages - leverages) * alpha
    gradients = np.exp(gradients)
    slopes *= gradients

    
# print info
np.set_printoptions(formatter=dict(float=lambda num: f"{num:5.2f}"))
print("payouts:   ", payouts)
print("leverages: ", leverages)
print("total pot: ", sum(payouts))

# print("gradients: ", gradients)
print("slopes:    ", slopes)

payouts:    [ 3.60  3.60  3.60  3.60  3.60  3.60]
leverages:  [ 1.50  1.50  1.50  1.50  1.50  1.50]
total pot:  21.601700516427453
slopes:     [ 0.35  0.35  0.35  0.35  0.35  0.35]
